In [ ]:
import tweepy
from textblob import TextBlob
import pandas as pd
import re
from datetime import datetime

# Replace these with your actual X API credentials
API_KEY = "api_key"
API_SECRET = "_api_secret"
ACCESS_TOKEN = "_access_token"
ACCESS_TOKEN_SECRET = "access_token_secret"
BEARER_TOKEN = "_bearer_token"

# Authenticate with X API v2
client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    consumer_key=API_KEY,
    consumer_secret=API_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET
)

# List of US stocks with their X cashtags
stocks = {
    "AAPL": "Apple",
    "MSFT": "Microsoft",
    "GOOGL": "Google",
    "TSLA": "Tesla",
    "AMZN": "Amazon"
}

# Function to clean text
def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)  # Remove special characters
    return text.strip()

# Function to get sentiment score
def get_sentiment(text):
    analysis = TextBlob(text)
    # Polarity ranges from -1 (negative) to 1 (positive)
    return analysis.sentiment.polarity

# Function to fetch and analyze posts for a stock
def analyze_stock_sentiment(stock_symbol, company_name, num_posts=100):
    query = f"${stock_symbol} -is:retweet lang:en"  # Search for cashtag, exclude retweets, English only
    try:
        # Fetch recent posts using X API v2
        tweets = client.search_recent_tweets(
            query=query,
            max_results=num_posts,
            tweet_fields=["created_at", "text"]
        )

        if not tweets.data:
            return {"stock": stock_symbol, "company": company_name, "average_sentiment": 0, "post_count": 0}

        # Process posts
        sentiments = []
        for tweet in tweets.data:
            cleaned_text = clean_text(tweet.text)
            sentiment = get_sentiment(cleaned_text)
            sentiments.append(sentiment)

        avg_sentiment = sum(sentiments) / len(sentiments)
        return {
            "stock": stock_symbol,
            "company": company_name,
            "average_sentiment": avg_sentiment,
            "post_count": len(sentiments)
        }

    except Exception as e:
        print(f"Error fetching data for {stock_symbol}: {e}")
        return {"stock": stock_symbol, "company": company_name, "average_sentiment": 0, "post_count": 0}

# Main function to run sentiment analysis
def main():
    results = []
    for symbol, name in stocks.items():
        print(f"Analyzing sentiment for {name} (${symbol})...")
        result = analyze_stock_sentiment(symbol, name)
        results.append(result)

    # Create a DataFrame for results
    df = pd.DataFrame(results)

    # Display results
    print("\nSentiment Analysis Results (as of", datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "):")
    print(df)

    # Save to CSV
    df.to_csv("stock_sentiment_analysis.csv", index=False)
    print("\nResults saved to 'stock_sentiment_analysis.csv'")

if __name__ == "__main__":
    main()